In [1]:
import torch
from torch import optim
from torch import nn
from pytorch_pretrained_bert import TransfoXLModel, TransfoXLCorpus, TransfoXLTokenizer
import time, math, sys
sys.path.append('../')
import numpy as np

import models
from utils.data_loader import JSONFileDataLoader
from utils.transf_data_loader import JSONFileDataLoaderTransf
from utils.framework import FewShotREFramework, FewShotREModel
from models.sentence_encoding.basic_sentence_encoder import CNNSentenceEncoder
from models.proto_model import Proto

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

from openai_transformer.model_pytorch import TransformerModel
from openai_transformer.model_pytorch import load_openai_pretrained_model, DEFAULT_CONFIG
from openai_transformer.text_utils import TextEncoder

import json

In [2]:
class TransformerSentenceEncoder(nn.Module):
    def __init__(self, n_special, n_ctx=512):
        nn.Module.__init__(self)
        self.args = DEFAULT_CONFIG
        self.model = TransformerModel(self.args, vocab=40990 + n_special, n_ctx=n_ctx)
        load_openai_pretrained_model(self.model, path='../openai_transformer/model/',
                                           path_names='../openai_transformer/',
                                           n_special=n_special, n_ctx=n_ctx)

    def forward(self, inputs):
        h = self.model(inputs)
        return h[-1].squeeze(-1)

In [3]:
model_name = 'proto'
N = 5
K = 5
max_length = 40
train_data_loader = JSONFileDataLoaderTransf('../data/fewrel_train.json',
                                             max_length=max_length, cuda=True)
val_data_loader = JSONFileDataLoaderTransf('../data/fewrel_val.json',
                                           max_length=max_length, cuda=True)

Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing
Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing


In [4]:
framework = FewShotREFramework(train_data_loader, val_data_loader, None)
encoder = TransformerSentenceEncoder(n_special=5)
model = Proto(encoder, hidden_size=768)

Loading weights...


In [ ]:
# %%time
# support, query, label = train_data_loader.next_batch(8, 5, 5, 5)
# encoder(support)

In [ ]:
# text_encoder = json.load(open('../finetune_transformer_lm/model/encoder_bpe_40000.json'))

In [ ]:
framework.train(model, 'proto', 4, 20, N, K, 5, val_step=500,
                optimizer=optim.Adam, learning_rate=0.1)

Start training...
cuda
step:  500 | loss: 3.046175, accuracy: 6.50%%
[EVAL] step: 1000 | accuracy: 20.04%
Best checkpoint
step: 1000 | loss: 3.088883, accuracy: 3.75%%
